In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
!git clone https://github.com/KanghwaSisters/24_2_mainSession.git

fatal: destination path '24_2_mainSession' already exists and is not an empty directory.


In [67]:
import os
os.chdir('/content/24_2_mainSession/4주차/env')

In [68]:
! python GridWorldEnvironment.py

In [69]:
from GridWorldEnvironment import GridWorldEnvironment

In [70]:
env = GridWorldEnvironment(start_point=(0,0),
                           end_point=(4,4),
                           gridworld_size=(5,5))

# Deep SARSA Class

In [82]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import pylab

# 딥살사 인공신경망(입력값: 상태 / 출력: 각 행동에 대한 큐함수)
class DeepSARSA(nn.Module):
    def __init__(self, state_size, action_size):
        super(DeepSARSA, self).__init__()
        self.fc1 = nn.Linear(state_size, 30)  # 입력층
        self.fc2 = nn.Linear(30, 30)  # 은닉층
        self.fc_out = nn.Linear(30, action_size)  # 출력층

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        q = self.fc_out(x)
        return q

# 그리드월드 예제에서의 딥살사 에이전트
class DeepSARSAgent:
    def __init__(self, state_size, action_size):
        # 상태의 크기와 행동의 크기 정의
        self.state_size = state_size
        self.action_size = action_size

        # 딥살사 하이퍼 파라미터
        self.discount_factor = 0.99 #할인율
        self.learning_rate = 0.001
        self.epsilon = 1.  #e-탐욕 정책: 처음에는 무조건 탐험한다
        self.epsilon_decay = .9999 #엡실론 값을 점차 낮춘다
        self.epsilon_min = 0.01

        #인공신경망 생성
        self.model = DeepSARSA(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), learning_rate=self.learning_rate)

    # e-탐욕 정책에 따라서 행동을 반환
    def get_action(self, state):
        if np.random.rand() <= self.epsilon: #0~1 사이 무작위 값과 엡실론 값 비교
            return random.randrange(self.action_size) #엡실론 값이 더 크다면 무작위(임의의) 행동 선택
        else: #무작위 값이 더 크다면 탐욕적인 행동 선택
            state = torch.FloatTensor(state)
            q_values = self.model(state) #현재 state에 대한 큐함수 값 출력
            return torch.argmax(q_values).item() #이 큐함수 값 중 가장 큰 값을 가지는 행동 반환

    # <s, a, r, s', a'>의 샘플로부터 인공신경망 업데이트
    def train_model(self, state, action, reward, next_state, next_action, done):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

        #학습 파라미터
        model_params=self.model.trainable_variables

        # 학습 파라미터- 상태와 다음 상태를 텐서로 변환
        next_state = torch.FloatTensor(next_state)

        predict = q_values[0] #예측/ 현재 상태에서의 예측 큐함수 값
        one_hot_action = F.one_hot(torch.tensor(action), self.action_size) # 실제로 한 행동에 해당하는 값만 추출하기 위해
        predict = torch.sum(one_hot_action*predict, axis=1) # 실제 행동에 대한 모델의 출력만

        # done = True 일 경우 에피소드가 끝나서 다음 상태가 없음
        with torch.no_grad():
          next_q_values = self.model(next_state)
          next_q = next_q_values[0][next_action] #다음 상태에서 다음 행동을 취했을 때 큐함수 값
        target = reward + (1 - done) * self.discount_factor * next_q #정답/ done=True일 경우 target=reward


        # MSE 오류함수 계산
        loss = nn.MSE(predict, torch.FloatTensor([target]))

        # 역전파 및 가중치 업데이트
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

# Main

- **Deep SARSA**를 이용해 그리드 월드 학습시키기  
- 학습 지표 시각화

In [84]:
if __name__ == "__main__": #환경과 에이전트가 상호작용
    # 환경 생성
    env = GridWorldEnvironment(start_point=(0,0),
                           end_point=(4,4),
                           gridworld_size=(5,5))
    state_size= env.state_len # 상태 공간 크기
    action_size= env.action_space # 행동 공간의 크기를 정수로 변환

    #에이전트 생성
    agent = DeepSARSAgent(state_size, action_size)

    scores = [] # 각 에피소드의 총 보상을 기록할 리스트

    for e in range(1000):
        done = False
        state = env.reset() # 환경 초기화
        state = np.identity(state_size)[state:state + 1]  # 상태를 one-hot 인코딩으로 변환
        score = 0 # 에피소드의 총 보상을 초기화

        while not done:
            # 현재 상태에 대한 행동 선택
            action = agent.get_action(state)

            # 선택한 행동으로 환경에서 한 타임스텝 진행 후 샘플 수집
            next_state, reward, done = env.step(action)
            next_state = np.identity(state_size)[next_state:next_state + 1]  # 다음 상태 변환
            next_action = agent.get_action(next_state) #다음 상태에 대해서 다음 행동을 구함

            # 샘플로 모델 학습
            agent.train_model(state, action, reward, next_state,
                                next_action, done) #살사니까 다음 상태의 다음 행동까지 다 고려해서 모델 학습
            score += reward #보상 누적
            state = next_state

            if done:
                # 에피소드마다 학습 결과 출력
                print("episode: {:3d} | score: {:3d} | epsilon: {:.3f}".format(
                      e + 1, score, agent.epsilon))
                scores.append(score) #에피소드 당 총 보상 저장
                episodes.append(e + 1)

                #학습 성과를 시각화
                pylab.plot(episodes, scores, 'b')
                pylab.xlabel("episode")
                pylab.ylabel("score")
                plt.title('Score per Episode')
                plt.show()


        # 100 에피소드마다 모델 저장
        if e % 100 == 0:
            torch.save(agent.model.state_dict(), 'save_model/model_{}.pth'.format(e))
            print("Model saved at episode: {}".format(e))

TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, torch.memory_format memory_format = None, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
